In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
! pip install xlrd # to handle Excel files

! pip install scipy
! pip install rpy2

import scipy
from scipy.stats.stats import pearsonr

from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector

stats = importr('stats')

import csv

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1nrnfi0fdyKMMpcYc9PEI-iAe1hbjILfb'
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('Achilles_gene_effect_2020q4.csv')
depmap = pd.read_csv("Achilles_gene_effect_2020q4.csv", sep=',')

depmap.head()

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),AADAC (13),AADACL2 (344752),AADACL3 (126767),AADACL4 (343066),AADAT (51166),AAED1 (195827),AAGAB (79719),AAK1 (22848),AAMDC (28971),AAMP (14),AANAT (15),AAR2 (25980),AARD (441376),AARS (16),AARS2 (57505),AARSD1 (80755),AASDH (132949),AASDHPPT (60496),AASS (10157),AATF (26574),AATK (9625),ABAT (18),ABCA1 (19),ABCA10 (10349),ABCA12 (26154),ABCA13 (154664),ABCA2 (20),ABCA3 (21),ABCA4 (24),ABCA5 (23461),...,ZRANB3 (84083),ZRSR2 (8233),ZSCAN1 (284312),ZSCAN10 (84891),ZSCAN16 (80345),ZSCAN18 (65982),ZSCAN2 (54993),ZSCAN20 (7579),ZSCAN21 (7589),ZSCAN22 (342945),ZSCAN23 (222696),ZSCAN25 (221785),ZSCAN29 (146050),ZSCAN30 (100101467),ZSCAN31 (64288),ZSCAN32 (54925),ZSCAN4 (201516),ZSCAN5A (79149),ZSCAN5B (342933),ZSCAN9 (7746),ZSWIM1 (90204),ZSWIM2 (151112),ZSWIM3 (140831),ZSWIM4 (65249),ZSWIM5 (57643),ZSWIM6 (57688),ZSWIM7 (125150),ZSWIM8 (23053),ZUP1 (221302),ZW10 (9183),ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000004,0.181332,0.089101,-0.193867,-0.024587,0.038458,-0.181824,0.351672,-0.440073,0.292582,0.167482,0.031556,0.044580,0.144478,0.177762,0.239003,-0.024592,-0.031614,0.011863,-0.865281,-0.276844,-0.455086,-0.093534,-1.797251,-0.689539,-0.125597,0.001550,-0.597598,0.067281,-0.529554,0.006442,-0.079719,0.100450,0.324237,-0.050694,0.044477,0.049399,-0.096716,-0.112030,0.191731,...,-0.059978,NaN,0.029713,0.115004,-0.018408,0.067082,-0.207863,-0.072436,-0.030453,-0.004016,0.228645,-0.072810,-0.230158,0.194470,0.038202,-0.141637,0.246798,-0.402616,-0.339199,0.045304,-0.414034,0.129288,-0.031113,0.168107,-0.364426,-0.040659,-0.181798,-0.166369,-0.159741,-0.165968,-0.124218,-0.469306,NaN,NaN,0.257361,0.244834,-0.408512,0.284734,0.226412,-0.149566
1,ACH-000005,-0.076383,0.245190,0.191238,0.153008,-0.197035,-0.323295,0.252522,-0.571498,-0.068945,0.011962,0.270155,0.123062,-0.093898,0.204735,0.212660,0.016289,-0.137302,-0.157561,-0.863007,-0.004683,-0.298031,0.093055,-2.025345,-0.543569,-0.240376,0.036181,-0.449487,0.154580,-0.658693,0.073690,-0.368808,-0.079149,0.313596,-0.057245,0.067510,0.108010,0.051102,0.041425,0.140282,...,0.144291,-2.059540,-0.011380,0.164447,0.115159,0.065833,-0.025208,-0.132683,-0.295381,-0.028137,0.126328,0.093528,-0.073895,0.026994,-0.023026,0.137047,0.264515,-0.294085,-0.517956,-0.005138,-0.015721,0.261385,-0.295973,-0.022354,-0.321764,0.198589,-0.258387,-0.265424,-0.240707,-0.523792,-0.212442,-0.426151,-0.068295,0.031635,0.205469,-0.068536,-0.092858,0.074640,0.028779,-0.264840
2,ACH-000007,0.102195,0.092449,-0.045926,0.171892,0.140561,0.170971,0.086060,-0.432320,0.010664,0.305225,0.035663,0.057404,0.090646,-0.019159,-0.030646,0.131583,-0.191657,0.127009,-1.001338,0.159676,0.250583,-0.150516,-1.637095,-0.760803,-0.208280,-0.361750,-0.689698,-0.164942,-0.682675,-0.028587,0.239511,-0.181307,0.334517,0.073738,0.192912,-0.122568,0.140142,0.054056,-0.091645,...,0.147319,-1.246032,0.098017,0.147173,-0.206661,-0.278335,-0.016862,-0.147785,-0.033742,0.063903,0.300092,-0.119422,0.146411,0.009710,0.140715,-0.184959,0.311851,0.040604,-0.472366,0.147080,-0.225732,-0.168598,0.205469,-0.005942,-0.232199,-0.108894,-0.155242,-0.100608,-0.105638,-0.597001,-0.083183,-0.269196,0.101146,0.277820,0.208814,0.001393,-0.327514,0.048714,-0.372854,-0.433157
3,ACH-000009,0.142342,-0.033126,-0.051224,0.060560,0.116002,-0.010044,0.104725,-0.610481,0.181508,0.096782,-0.110764,-0.228326,-0.062743,0.109000,-0.261947,0.134506,-0.252298,-0.128475,-0.900731,-0.284180,-0.078233,-0.054680,-1.545906,-0.702111,-0.417086,-0.090473,-0.641356,0.215492,-0.597993,-0.192084,0.184858,-0.128961,0.171489,-0.250804,-0.057680,-0.112663,0.017168,-0.075971,-0.333881,...,-0.045675,-0.980097,-0.179418,-0.067685,0.007080,-0.125727,-0.073767,-0.245966,0.007498,-0.036320,0.144672,-0.240805,-0.042349,-0.105993,0.103997,0.172302,0.045258,-0.06

In [2]:
file_id = '1hmbJBbDlqfSrq7YvhZlCKWwisFTeOZkH'
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('third-partypathogens_combined_top_hits_15screens_gt2_pathogens.csv')
all_pathogen_genes = pd.read_csv("third-partypathogens_combined_top_hits_15screens_gt2_pathogens.csv", sep=',')

all_pathogen_genes.head()

,gene
0,VPS53
1,COG3
2,SCFD1
3,COG4
4,VPS54


In [3]:
file_id = '1tPKF62wB0gAEjeoiO7Y0Yfl9-BYZpL4t'
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('primary-screen-replicate-collapsed-treatment-info.csv')
drugs = pd.read_csv("primary-screen-replicate-collapsed-treatment-info.csv", sep=',')

drugs.head()

,column_name,broad_id,name,dose,screen_id,moa,target,disease.area,indication,smiles,phase
0,BRD-A00055058-001-01-0::2.325889319::MTS004,BRD-A00055058-001-01-0,RS-0481,2.325889,MTS004,immunostimulant,NaN,NaN,NaN,CC(NC(=O)C1CSCN1C(=O)c1ccccc1)c1ccccc1,Phase 2
1,BRD-A00842753-001-01-9::2.5::MTS004,BRD-A00842753-001-01-9,oleuropein,2.500000,MTS004,estrogen receptor agonist,GPER1,NaN,NaN,COC(=O)C1=COC(OC2OC(CO)C(O)C(O)C2O)\C(=C/C)C1C...,Phase 2
2,BRD-A02232681-001-01-8::2.5::MTS004,BRD-A02232681-001-01-8,isoleucine,2.500000,MTS004,NaN,"ACADSB, BCAT1, BCAT2, IARS, IARS2",NaN,NaN,CCC(C)C(N)C(O)=O,Launched
3,BRD-A04447196-001-01-8::2.5::MTS004,BRD-A04447196-001-01-8,gepefrine,2.500000,MTS004,adrenergic receptor agonist,NaN,cardiology,hypotension,CC(N)Cc1cccc(O)c1,Launched
4,BRD-A04971881-003-01-3::2.65294603::MTS004,BRD-A04971881-003-01-3,cloranolol,2.652946,MTS004,adrenergic receptor antagonist,"ADRB1, ADRB2, ADRB3",NaN,NaN,CC(C)(C)NCC(O)COc1cc(Cl)ccc1Cl,Launched


In [4]:
file_id = '1LgLjrInt-su5dhQRpm2LURVOZ0mzfBLy'
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('primary_screen_replicate_collapsed_logfold_change.csv')
screen = pd.read_csv("primary_screen_replicate_collapsed_logfold_change.csv", sep=',')

screen.head()

,Unnamed: 0,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
0,ACH-000001,-0.015577,-0.449332,0.489379,0.206675,0.272730,0.021036,-0.025460,0.467158,-0.736306,0.644137,-0.371126,0.425098,NaN,0.287026,0.410469,0.404994,0.702347,-0.648735,0.537157,-0.051923,-0.463111,0.213471,0.513649,0.561612,0.092447,-0.840919,0.609556,-1.401019,0.774219,-1.411322,-0.139209,0.265538,0.648261,-0.382506,NaN,0.058709,-0.329350,0.227541,0.212912,...,-0.058402,0.387044,0.385874,0.400234,0.199965,0.005106,0.528592,0.014759,-0.070412,0.086858,0.027550,0.024090,-0.051964,0.160257,-0.248010,0.133490,0.283482,0.000754,-0.007040,0.448325,-0.091233,-0.063261,-0.331619,0.671945,0.200060,0.256095,0.154881,0.383746,0.032667,-0.087383,0.429238,0.204841,0.150055,-0.575404,-0.101247,0.399233,-0.127658,-0.141651,-1.153652,0.510464
1,ACH-000007,-0.095730,0.257943,0.772349,-0.438502,-0.732832,0.779201,0.426523,-1.288508,-0.476133,-0.277105,-1.249108,-0.525292,0.400643,-0.732619,0.065891,-1.116199,0.527021,-1.631676,-0.579875,0.556004,0.423156,0.176780,0.192204,-0.015698,0.124954,-0.364671,0.239814,-0.291710,-0.131295,-2.565264,-0.365848,-0.21

In [6]:
depmap.shape
drugs.shape
screen.shape
all_pathogen_genes.shape

(808, 18120)

(4686, 11)

(578, 4687)

(171, 1)

In [7]:
depmap_ = depmap

In [120]:
depmap = depmap_

In [8]:
drugs_ = drugs

In [121]:
drugs = drugs_

In [9]:
screen_ = screen

In [122]:
screen = screen_

In [10]:
all_pathogen_genes_ = all_pathogen_genes

In [123]:
all_pathogen_genes = all_pathogen_genes_

In [124]:
depmap.columns = list(map(lambda s: s.split()[0], depmap.columns))

depmap.head()

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ACH-000004,0.181332,0.089101,-0.193867,-0.024587,0.038458,-0.181824,0.351672,-0.440073,0.292582,0.167482,0.031556,0.044580,0.144478,0.177762,0.239003,-0.024592,-0.031614,0.011863,-0.865281,-0.276844,-0.455086,-0.093534,-1.797251,-0.689539,-0.125597,0.001550,-0.597598,0.067281,-0.529554,0.006442,-0.079719,0.100450,0.324237,-0.050694,0.044477,0.049399,-0.096716,-0.112030,0.191731,...,-0.059978,NaN,0.029713,0.115004,-0.018408,0.067082,-0.207863,-0.072436,-0.030453,-0.004016,0.228645,-0.072810,-0.230158,0.194470,0.038202,-0.141637,0.246798,-0.402616,-0.339199,0.045304,-0.414034,0.129288,-0.031113,0.168107,-0.364426,-0.040659,-0.181798,-0.166369,-0.159741,-0.165968,-0.124218,-0.469306,NaN,NaN,0.257361,0.244834,-0.408512,0.284734,0.226412,-0.149566
1,ACH-000005,-0.076383,0.245190,0.191238,0.153008,-0.197035,-0.323295,0.252522,-0.571498,-0.068945,0.011962,0.270155,0.123062,-0.093898,0.204735,0.212660,0.016289,-0.137302,-0.157561,-0.863007,-0.004683,-0.298031,0.093055,-2.025345,-0.543569,-0.240376,0.036181,-0.449487,0.154580,-0.658693,0.073690,-0.368808,-0.079149,0.313596,-0.057245,0.067510,0.108010,0.051102,0.041425,0.140282,...,0.144291,-2.059540,-0.011380,0.164447,0.115159,0.065833,-0.025208,-0.132683,-0.295381,-0.028137,0.126328,0.093528,-0.073895,0.026994,-0.023026,0.137047,0.264515,-0.294085,-0.517956,-0.005138,-0.015721,0.261385,-0.295973,-0.022354,-0.321764,0.198589,-0.258387,-0.265424,-0.240707,-0.523792,-0.212442,-0.426151,-0.068295,0.031635,0.205469,-0.068536,-0.092858,0.074640,0.028779,-0.264840
2,ACH-000007,0.102195,0.092449,-0.045926,0.171892,0.140561,0.170971,0.086060,-0.432320,0.010664,0.305225,0.035663,0.057404,0.090646,-0.019159,-0.030646,0.131583,-0.191657,0.127009,-1.001338,0.159676,0.250583,-0.150516,-1.637095,-0.760803,-0.208280,-0.361750,-0.689698,-0.164942,-0.682675,-0.028587,0.239511,-0.181307,0.334517,0.073738,0.192912,-0.122568,0.140142,0.054056,-0.091645,...,0.147319,-1.246032,0.098017,0.147173,-0.206661,-0.278335,-0.016862,-0.147785,-0.033742,0.063903,0.300092,-0.119422,0.146411,0.009710,0.140715,-0.184959,0.311851,0.040604,-0.472366,0.147080,-0.225732,-0.168598,0.205469,-0.005942,-0.232199,-0.108894,-0.155242,-0.100608,-0.105638,-0.597001,-0.083183,-0.269196,0.101146,0.277820,0.208814,0.001393,-0.327514,0.048714,-0.372854,-0.433157
3,ACH-000009,0.142342,-0.033126,-0.051224,0.060560,0.116002,-0.010044,0.104725,-0.610481,0.181508,0.096782,-0.110764,-0.228326,-0.062743,0.109000,-0.261947,0.134506,-0.252298,-0.128475,-0.900731,-0.284180,-0.078233,-0.054680,-1.545906,-0.702111,-0.417086,-0.090473,-0.641356,0.215492,-0.597993,-0.192084,0.184858,-0.128961,0.171489,-0.250804,-0.057680,-0.112663,0.017168,-0.075971,-0.333881,...,-0.045675,-0.980097,-0.179418,-0.067685,0.007080,-0.125727,-0.073767,-0.245966,0.007498,-0.036320,0.144672,-0.240805,-0.042349,-0.105993,0.103997,0.172302,0.045258,-0.067205,-0.365005,0.179872,-0.026404,-0.128260,0.022729,0.121146,-0.300059,0.253921,-0.380699,-0.176251,0.100350,-0.452592,-0.277264,-0.307018,0.044741,0.201551,0.083866,0.052208,-0.574719,0.218682,-0.074750,-0.551760
4,ACH-000011,0.280082,0.088898,0.032321,0.446598,-0.037188,-0.228207,0.110942,-0.406541,0.153979,0.109069,0.192735,-0.102440,-0.097932,0.062355,-0.208635,0.056466,-0.266804,0.000690,-0.934304,-0.257453,0.222780,-0.326677,-1.570667,-0.395234,-0.441706,-0.135648,-0.634220,0.154027,-0.644039,-0.325782,-0.061227,0.153304,0.068585,-0.172936,0.041381,0.087714,-0.231762,0.010596,-0.076289,...,-0.069146,

In [125]:
screen = screen.rename(columns = {'Unnamed: 0' : 'DepMap_ID'}) # 2020q2

#screen = screen.set_index('DepMap_ID')

screen.head()


,DepMap_ID,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
0,ACH-000001,-0.015577,-0.449332,0.489379,0.206675,0.272730,0.021036,-0.025460,0.467158,-0.736306,0.644137,-0.371126,0.425098,NaN,0.287026,0.410469,0.404994,0.702347,-0.648735,0.537157,-0.051923,-0.463111,0.213471,0.513649,0.561612,0.092447,-0.840919,0.609556,-1.401019,0.774219,-1.411322,-0.139209,0.265538,0.648261,-0.382506,NaN,0.058709,-0.329350,0.227541,0.212912,...,-0.058402,0.387044,0.385874,0.400234,0.199965,0.005106,0.528592,0.014759,-0.070412,0.086858,0.027550,0.024090,-0.051964,0.160257,-0.248010,0.133490,0.283482,0.000754,-0.007040,0.448325,-0.091233,-0.063261,-0.331619,0.671945,0.200060,0.256095,0.154881,0.383746,0.032667,-0.087383,0.429238,0.204841,0.150055,-0.575404,-0.101247,0.399233,-0.127658,-0.141651,-1.153652,0.510464
1,ACH-000007,-0.095730,0.257943,0.772349,-0.438502,-0.732832,0.779201,0.426523,-1.288508,-0.476133,-0.277105,-1.249108,-0.525292,0.400643,-0.732619,0.065891,-1.116199,0.527021,-1.631676,-0.579875,0.556004,0.423156,0.176780,0.192204,-0.015698,0.124954,-0.364671,0.239814,-0.291710,-0.131295,-2.565264,-0.365848,-0.212

In [126]:
screen_rank = screen.rank()

screen_rank.head()

,DepMap_ID,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
0,1.0,276.0,140.0,498.0,253.0,510.0,131.0,158.0,545.0,338.0,522.0,37.0,472.0,NaN,523.0,367.0,511.0,533.0,199.0,524.0,371.0,82.0,255.0,554.0,549.0,169.0,73.0,553.0,28.0,527.0,550.0,539.0,339.0,527.0,66.0,NaN,182.0,182.0,371.0,274.0,...,316.0,486.0,495.0,492.0,330.0,275.0,500.0,339.0,158.0,319.0,326.0,299.0,280.0,397.0,85.0,328.0,452.0,227.0,171.0,535.0,202.0,261.0,161.0,539.0,406.0,380.0,456.0,473.0,243.0,225.0,439.0,393.0,361.0,79.0,155.0,498.0,227.0,198.0,509.0,504.0
1,2.0,235.0,511.0,547.0,19.0,28.0,535.0,453.0,20.0,445.0,29.0,1.0,37.0,491.0,59.0,151.0,6.0,485.0,57.0,30.0,561.0,527.0,232.0,514.0,291.0,191.0,375.0,446.0,322.0,77.0,471.0,499.0,99.0,518.0,530.0,541.0,490.0,362.0,503.0,307.0,...,36.0,535.0,532.0,394.0,412.0,447.0,531.0,523.0,548.0,500.0,523.0,275.0,417.0,537.0,412.0,415.0,378.0,520.0,510.0,381.0,353.0,445.0,287.0,454.0,531.0,166.0,136.0,478.0,373.0,274.0,14.0,398.0,179.0,542.0,473.0,410.0,491.0,93.0,529.0,426.0
2,3.0,455.0,84.0,516.0,394.0,247.0,161.0,275.0,196.0,430.0,451.0,349.0,355.0,274.0,425.0,407.0,323.0,193.0,376.0,124.0,84.

In [127]:
take_top_X_genes = 50
take_top_X_genes = 250

In [128]:
screen_rank_lt50 = screen_rank.applymap(lambda x: 1 if ( x < take_top_X_genes) else 0 ) # (NaN if (pd.isnull(x)) else 0)

screen_rank_lt50.head()

,DepMap_ID,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0
1,1,1,0,0,1,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0
2,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,...,0,0,1,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1
3,1,0,0,0,1,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,1,0,1,0,0,...,1,0,0,1,1,1,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1
4,1,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0


In [129]:
screen_rank_lt50['DepMap_ID'] = screen_['Unnamed: 0']
screen_rank_lt50.shape
screen_rank_lt50.head()

m = screen_rank_lt50['DepMap_ID'].isin(depmap['DepMap_ID'])
screen_rank_lt50 = screen_rank_lt50[m]

#del screen_rank_lt50['DepMap_ID']

screen_rank_lt50.shape
screen_rank_lt50.head()

(578, 4687)

,DepMap_ID,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
0,ACH-000001,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,1,0,1,1,0,0
1,ACH-000007,1,0,0,1,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0
2,ACH-000008,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,...,0,0,1,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,1
3,ACH-000010_FAILED_STR,0,0,0,1,1,1,1,1,0,0,1,1,0,1,1,0,1,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,1,1,0,1,0,0,...,1,0,0,1,1,1,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1
4,ACH-000011,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0


(404, 4687)

,DepMap_ID,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
1,ACH-000007,1,0,0,1,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0
4,ACH-000011,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0
5,ACH-000012,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,1,0
6,ACH-000013,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
7,ACH-000014,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0


In [130]:
screen_rank_lt50_sums = screen_rank_lt50.sum(axis=1) 

screen_rank_lt50_sums.head()

1    2233
4    1990
5    1833
6    1520
7    2264
dtype: int64

In [131]:
depmap_rank = depmap.rank()

depmap.head()
depmap_rank.head()

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ACH-000004,0.181332,0.089101,-0.193867,-0.024587,0.038458,-0.181824,0.351672,-0.440073,0.292582,0.167482,0.031556,0.044580,0.144478,0.177762,0.239003,-0.024592,-0.031614,0.011863,-0.865281,-0.276844,-0.455086,-0.093534,-1.797251,-0.689539,-0.125597,0.001550,-0.597598,0.067281,-0.529554,0.006442,-0.079719,0.100450,0.324237,-0.050694,0.044477,0.049399,-0.096716,-0.112030,0.191731,...,-0.059978,NaN,0.029713,0.115004,-0.018408,0.067082,-0.207863,-0.072436,-0.030453,-0.004016,0.228645,-0.072810,-0.230158,0.194470,0.038202,-0.141637,0.246798,-0.402616,-0.339199,0.045304,-0.414034,0.129288,-0.031113,0.168107,-0.364426,-0.040659,-0.181798,-0.166369,-0.159741,-0.165968,-0.124218,-0.469306,NaN,NaN,0.257361,0.244834,-0.408512,0.284734,0.226412,-0.149566
1,ACH-000005,-0.076383,0.245190,0.191238,0.153008,-0.197035,-0.323295,0.252522,-0.571498,-0.068945,0.011962,0.270155,0.123062,-0.093898,0.204735,0.212660,0.016289,-0.137302,-0.157561,-0.863007,-0.004683,-0.298031,0.093055,-2.025345,-0.543569,-0.240376,0.036181,-0.449487,0.154580,-0.658693,0.073690,-0.368808,-0.079149,0.313596,-0.057245,0.067510,0.108010,0.051102,0.041425,0.140282,...,0.144291,-2.059540,-0.011380,0.164447,0.115159,0.065833,-0.025208,-0.132683,-0.295381,-0.028137,0.126328,0.093528,-0.073895,0.026994,-0.023026,0.137047,0.264515,-0.294085,-0.517956,-0.005138,-0.015721,0.261385,-0.295973,-0.022354,-0.321764,0.198589,-0.258387,-0.265424,-0.240707,-0.523792,-0.212442,-0.426151,-0.068295,0.031635,0.205469,-0.068536,-0.092858,0.074640,0.028779,-0.264840
2,ACH-000007,0.102195,0.092449,-0.045926,0.171892,0.140561,0.170971,0.086060,-0.432320,0.010664,0.305225,0.035663,0.057404,0.090646,-0.019159,-0.030646,0.131583,-0.191657,0.127009,-1.001338,0.159676,0.250583,-0.150516,-1.637095,-0.760803,-0.208280,-0.361750,-0.689698,-0.164942,-0.682675,-0.028587,0.239511,-0.181307,0.334517,0.073738,0.192912,-0.122568,0.140142,0.054056,-0.091645,...,0.147319,-1.246032,0.098017,0.147173,-0.206661,-0.278335,-0.016862,-0.147785,-0.033742,0.063903,0.300092,-0.119422,0.146411,0.009710,0.140715,-0.184959,0.311851,0.040604,-0.472366,0.147080,-0.225732,-0.168598,0.205469,-0.005942,-0.232199,-0.108894,-0.155242,-0.100608,-0.105638,-0.597001,-0.083183,-0.269196,0.101146,0.277820,0.208814,0.001393,-0.327514,0.048714,-0.372854,-0.433157
3,ACH-000009,0.142342,-0.033126,-0.051224,0.060560,0.116002,-0.010044,0.104725,-0.610481,0.181508,0.096782,-0.110764,-0.228326,-0.062743,0.109000,-0.261947,0.134506,-0.252298,-0.128475,-0.900731,-0.284180,-0.078233,-0.054680,-1.545906,-0.702111,-0.417086,-0.090473,-0.641356,0.215492,-0.597993,-0.192084,0.184858,-0.128961,0.171489,-0.250804,-0.057680,-0.112663,0.017168,-0.075971,-0.333881,...,-0.045675,-0.980097,-0.179418,-0.067685,0.007080,-0.125727,-0.073767,-0.245966,0.007498,-0.036320,0.144672,-0.240805,-0.042349,-0.105993,0.103997,0.172302,0.045258,-0.067205,-0.365005,0.179872,-0.026404,-0.128260,0.022729,0.121146,-0.300059,0.253921,-0.380699,-0.176251,0.100350,-0.452592,-0.277264,-0.307018,0.044741,0.201551,0.083866,0.052208,-0.574719,0.218682,-0.074750,-0.551760
4,ACH-000011,0.280082,0.088898,0.032321,0.446598,-0.037188,-0.228207,0.110942,-0.406541,0.153979,0.109069,0.192735,-0.102440,-0.097932,0.062355,-0.208635,0.056466,-0.266804,0.000690,-0.934304,-0.257453,0.222780,-0.326677,-1.570667,-0.395234,-0.441706,-0.135648,-0.634220,0.154027,-0.644039,-0.325782,-0.061227,0.153304,0.068585,-0.172936,0.041381,0.087714,-0.231762,0.010596,-0.076289,...,-0.069146,

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1.0,596.0,461.0,69.0,40.0,589.0,92.0,785.0,523.0,782.0,426.0,241.0,575.0,742.0,718.0,792.0,319.0,791.0,520.0,299.0,179.0,11.0,704.0,210.0,509.0,661.0,519.0,346.0,179.0,673.0,752.0,362.0,678.0,678.0,487.0,597.0,553.0,238.0,392.0,798.0,...,106.0,NaN,656.0,781.0,464.0,737.0,180.0,660.0,611.0,333.0,682.0,754.0,26.0,784.0,476.0,89.0,490.0,21.0,229.0,336.0,15.0,642.0,126.0,755.0,37.0,187.0,675.0,614.0,59.0,797.0,374.0,520.0,NaN,NaN,721.0,753.0,37.0,737.0,800.0,740.0
1,2.0,51.0,753.0,797.0,310.0,86.0,13.0,726.0,268.0,86.0,79.0,755.0,717.0,208.0,747.0,785.0,402.0,744.0,103.0,302.0,768.0,41.0,799.0,29.0,677.0,393.0,607.0,566.0,469.0,481.0,784.0,14.0,205.0,655.0,470.0,641.0,693.0,636.0,736.0,784.0,...,633.0,5.0,544.0,792.0,743.0,734.0,658.0,498.0,70.0,264.0,390.0,805.0,175.0,630.0,243.0,699.0,559.0,88.0,23.0,185.0,608.0,782.0,2.0,358.0,69.0,692.0,537.0,406.0,24.0,558.0,197.0,577.0,214.0,316.0,654.0,80.0,639.0,249.0,748.0,627.0
2,3.0,370.0,473.0,444.0,368.0,744.0,773.0,302.0,536.0,201.0,724.0,249.0,609.0,681.0,153.0,427.0,652.0,671.0,722.0,113.0,806.0,796.0,598.0,489.0,413.0,477.0,11.0,217.0,11.0,436.0,718.0,781.0,62.0,697.0,731.0,781.0,126.0,753.0,750.0,367.0,...,643.0,371.0,757.0,790.0,47.0,64.0,678.0,452.0,600.0,549.0,768.0,708.0,717.0,578.0,753.0,55.0,678.0,748.0,47.0,637.0,122.0,33.0,710.0,408.0,197.0,96.0,708.0,699.0,115.0,457.0,458.0,705.0,576.0,734.0,659.0,205.0,104.0,200.0,70.0,282.0
3,4.0,503.0,141.0,425.0,113.0,721.0,507.0,365.0,208.0,676.0,210.0,57.0,28.0,272.0,583.0,29.0,656.0,535.0,154.0,234.0,162.0,309.0,748.0,641.0,494.0,65.0,243.0,276.0,634.0,587.0,336.0,767.0,111.0,181.0,80.0,279.0,139.0,558.0,519.0,21.0,...,134.0,688.0,131.0,501.0,542.0,337.0,541.0,196.0,680.0,245.0,459.0,397.0,241.0,245.0,684.0,742.0,51.0,580.0,180.0,702.0,577.0,60.0,238.0,714.0,93.0,747.0,259.0,590.0,684.0,655.0,103.0,680.0,444.0,666.0,337.0,338.0,6.0,624.0,625.0,119.0
4,5.0,757.0,460.0,664.0,792.0,395.0,56.0,381.0,580.0,609.0,240.0,626.0,154.0,200.0,434.0,63.0,510.0,492.0,487.0,194.0,234.0,789.0,158.0,613.0,768.0,55.0,150.0,283.0,465.0,506.0,107.0,401.0,748.0,31.0,169.0,589.0,652.0,46.0,705.0,399.0,...,94.0,268.0,780.0,48.0,581.0,447.0,57.0,167.0,507.0,653.0,729.0,372.0,750.0,678.0,280.0,36.0,128.0,740.0,462.0,211.0,368.0,86.0,576.0,796.0,126.0,149.0,314.0,708.0,627.0,312.0,38.0,511.0,352.0,279.0,745.0,582.0,245.0,349.0,235.0,471.0


In [132]:
depmap_rank_lt50 = depmap_rank.applymap(lambda x: 1 if (x< take_top_X_genes) else 0 ) # (NaN if (pd.isnull(x)) else 0)

depmap_rank_lt50.head()

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,1,1,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0
2,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,1,1,0
3,1,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,0,0,1,...,1,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
4,1,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0


In [133]:
depmap_rank_lt50['DepMap_ID'] = depmap['DepMap_ID']


depmap_rank_lt50.shape
depmap_rank_lt50.head()

n = depmap_rank_lt50['DepMap_ID'].isin(screen_['Unnamed: 0'])
depmap_rank_lt50 = depmap_rank_lt50[n]

#del depmap_rank_lt50['DepMap_ID']

depmap_rank_lt50.shape
depmap_rank_lt50.head()

(808, 18120)

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ACH-000004,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1,ACH-000005,1,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,1,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0
2,ACH-000007,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,1,1,0
3,ACH-000009,0,1,0,1,0,0,0,1,0,1,1,1,0,0,1,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,1,0,0,1,...,1,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
4,ACH-000011,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0


(404, 18120)

,DepMap_ID,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
2,ACH-000007,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,1,1,0
4,ACH-000011,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,...,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0
5,ACH-000012,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,1,...,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0
6,ACH-000013,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1
7,ACH-000014,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0


In [134]:
depmap_rank_lt50_sums = depmap_rank_lt50.sum(axis=1) 

depmap_rank_lt50_sums.head()

2    5439
4    5397
5    4925
6    4229
7    5865
dtype: int64

In [135]:
depmap_rank_lt50_sums_sum = depmap_rank_lt50_sums.sum()

depmap_rank_lt50_sums_sum

2200126

In [136]:
pathogen_genes_existing = ['DepMap_ID']
for col in all_pathogen_genes['gene'].to_list():
    if col in depmap_rank_lt50.columns:
      pathogen_genes_existing.append(col)


In [137]:
pathogen_depmap = depmap_rank_lt50[pathogen_genes_existing]

pathogen_depmap.head()
pathogen_depmap.shape

,DepMap_ID,VPS53,COG3,SCFD1,COG4,VPS54,COG8,TM9SF2,COG2,VPS52,SPTLC2,SLC35B2,SLC35A2,COG7,WDR7,TMEM199,ATP6AP2,COG1,SPTSSA,UGCG,TRAPPC1,ATP6V1H,TMED2,A4GALT,GOLPH3,VPS51,ARL1,ATP6V0C,ATP6V1E1,ATP6V1A,ATP6V0D1,ATP6V1C1,RAB1A,RAB2A,ATP6V0B,VPS29,POLR3A,KDSR,VPS18,TRAPPC8,...,SACM1L,GALNTL6,SETDB1,SEC31A,ZNF281,DDX55,FBXO21,PI4KB,RPN2,ANKRD13C,CTCFL,INTS7,TRAPPC2L,CCDC115,DDX10,GEMIN2,RAD21,PIK3C3,ARF1,UPF2,CYCS,CENPT,BCORL1,TPR,EGFR,FLII,HIST1H3H,PREB,SUDS3,EEF2,METAP2,MPHOSPH6,DIAPH2,RPTOR,SLC25A26,URM1,ITPK1,VTI1B,TNPO2,AADAT
2,ACH-000007,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,...,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1
4,ACH-000011,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,0,...,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0,1,0,0
5,ACH-000012,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
6,ACH-000013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
7,ACH-000014,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,...,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1,1,0,0


(404, 167)

In [138]:
pathogen_depmap_sums = pathogen_depmap.sum(axis=1) 

pathogen_depmap_sums.head()

2    60
4    49
5    42
6    37
7    46
dtype: int64

In [139]:
pathogen_depmap_sums_sum = pathogen_depmap_sums.sum()

pathogen_depmap_sums_sum

19783

In [140]:
screen_rank_lt50_i = screen_rank_lt50.set_index('DepMap_ID')
screen_rank_lt50_i.head()

,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,BRD-A04497688-001-04-2::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
DepMap_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1,0,0,1,1,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,1,0,0
ACH-000011,0,1,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,...,1,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,0
ACH-000012,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,1,1,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,1,0,0,1,1,0,1,1,1,0,1,0,1,0,1,0,0,1,1,0
ACH-000013,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
ACH-000014,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,1,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0


In [141]:
#screen_rank_lt50_i = screen_rank_lt50.set_index('DepMap_ID')

depmap_rank_lt50_i = depmap_rank_lt50.set_index('DepMap_ID')
depmap_rank_lt50_i.head()
pathogen_depmap_i = pathogen_depmap.set_index('DepMap_ID')
pathogen_depmap_i.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,AADACL3,AADACL4,AADAT,AAED1,AAGAB,AAK1,AAMDC,AAMP,AANAT,AAR2,AARD,AARS,AARS2,AARSD1,AASDH,AASDHPPT,AASS,AATF,AATK,ABAT,ABCA1,ABCA10,ABCA12,ABCA13,ABCA2,ABCA3,ABCA4,ABCA5,ABCA6,...,ZRANB3,ZRSR2,ZSCAN1,ZSCAN10,ZSCAN16,ZSCAN18,ZSCAN2,ZSCAN20,ZSCAN21,ZSCAN22,ZSCAN23,ZSCAN25,ZSCAN29,ZSCAN30,ZSCAN31,ZSCAN32,ZSCAN4,ZSCAN5A,ZSCAN5B,ZSCAN9,ZSWIM1,ZSWIM2,ZSWIM3,ZSWIM4,ZSWIM5,ZSWIM6,ZSWIM7,ZSWIM8,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACH-000007,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,1,1,1,0
ACH-000011,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,0,0,1,...,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0
ACH-000012,1,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,1,0,...,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0
ACH-000013,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1
ACH-000014,0,1,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0


,VPS53,COG3,SCFD1,COG4,VPS54,COG8,TM9SF2,COG2,VPS52,SPTLC2,SLC35B2,SLC35A2,COG7,WDR7,TMEM199,ATP6AP2,COG1,SPTSSA,UGCG,TRAPPC1,ATP6V1H,TMED2,A4GALT,GOLPH3,VPS51,ARL1,ATP6V0C,ATP6V1E1,ATP6V1A,ATP6V0D1,ATP6V1C1,RAB1A,RAB2A,ATP6V0B,VPS29,POLR3A,KDSR,VPS18,TRAPPC8,TMED10,...,SACM1L,GALNTL6,SETDB1,SEC31A,ZNF281,DDX55,FBXO21,PI4KB,RPN2,ANKRD13C,CTCFL,INTS7,TRAPPC2L,CCDC115,DDX10,GEMIN2,RAD21,PIK3C3,ARF1,UPF2,CYCS,CENPT,BCORL1,TPR,EGFR,FLII,HIST1H3H,PREB,SUDS3,EEF2,METAP2,MPHOSPH6,DIAPH2,RPTOR,SLC25A26,URM1,ITPK1,VTI1B,TNPO2,AADAT
DepMap_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACH-000007,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,...,1,0,0,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1
ACH-000011,0,1,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,1,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,0,0,1,0,1,0,0
ACH-000012,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
ACH-000013,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
ACH-000014,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1,0,0,0,1,1,0,1,1,0,0,1,...,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1,1,0,0


In [ ]:
depmap_rank_lt50_sums_list = depmap_rank_lt50_sums.to_list()
depmap_rank_lt50_sums_list

In [ ]:
depmap_rank_lt50_sums_list = depmap_rank_lt50_sums.to_list()
depmap_rank_lt50_sums_list

In [ ]:
pathogen_depmap_sums_list = pathogen_depmap_sums.to_list()
pathogen_depmap_sums_list

In [145]:
screen_rank_lt50_all_mul = screen_rank_lt50_i.mul(pd.Series(depmap_rank_lt50_sums_list, index=screen_rank_lt50_i.index.to_list()), axis='index')

screen_rank_lt50_all_mul.head()
screen_rank_lt50_all_mul.shape

,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,BRD-A04497688-001-04-2::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
DepMap_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACH-000007,5439,0,0,5439,5439,0,0,5439,0,5439,5439,5439,0,5439,5439,5439,0,5439,5439,0,0,5439,0,0,5439,0,0,0,5439,0,0,5439,0,0,0,0,0,0,0,0,...,5439,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5439,5439,0,0,0,5439,0,5439,0,0,0,0,5439,0,0
ACH-000011,0,5397,0,5397,5397,0,5397,0,0,5397,0,0,5397,0,0,0,0,0,0,5397,5397,0,5397,5397,0,0,5397,5397,0,5397,0,0,5397,5397,0,0,5397,0,0,0,...,5397,0,0,0,5397,5397,5397,0,5397,0,0,0,0,5397,0,0,5397,0,5397,0,5397,5397,5397,5397,5397,0,0,0,0,0,5397,5397,0,5397,0,0,0,5397,5397,0
ACH-000012,0,0,0,0,0,0,4925,4925,0,4925,0,4925,0,0,4925,0,0,0,0,0,4925,4925,4925,0,0,0,4925,0,4925,0,4925,0,0,0,4925,0,0,0,0,0,...,0,0,4925,4925,0,0,4925,0,0,4925,0,4925,0,4925,0,0,0,0,0,4925,4925,0,0,4925,4925,0,4925,4925,4925,0,4925,0,4925,0,4925,0,0,4925,4925,0
ACH-000013,0,0,0,4229,0,0,4229,0,0,0,0,0,0,4229,4229,0,0,0,0,4229,4229,0,0,4229,4229,4229,0,0,4229,0,0,0,4229,0,0,0,0,0,4229,0,...,0,0,0,0,0,0,0,0,4229,0,0,0,0,4229,

(404, 4686)

In [146]:
screen_rank_lt50_pathogens_only_mul = screen_rank_lt50_i.mul(pd.Series(pathogen_depmap_sums_list, index=screen_rank_lt50_i.index.to_list()), axis='index')

screen_rank_lt50_pathogens_only_mul.head()
screen_rank_lt50_pathogens_only_mul.shape

,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00520476-001-07-4::2.5::HTS,BRD-A00546892-001-02-6::2.5::HTS,BRD-A00578795-001-04-3::2.5::HTS,BRD-A00758722-001-04-9::2.5::HTS,BRD-A00827783-001-24-6::2.5::HTS,BRD-A00993607-003-24-6::2.5::HTS,BRD-A01098288-001-02-9::2.5::HTS,BRD-A01412266-001-01-0::2.5::HTS,BRD-A01493904-003-12-1::2.5::HTS,BRD-A01563671-001-02-7::2.5::HTS,BRD-A01593789-001-03-1::2.5::HTS,BRD-A01636364-003-15-1::2.5::HTS,BRD-A01643550-001-04-9::2.5::HTS,BRD-A01787639-003-16-9::2.5::HTS,BRD-A01907367-001-01-7::2.5::HTS,BRD-A02006392-001-16-4::2.5::HTS,BRD-A02180903-001-04-5::2.5::HTS,BRD-A02710418-003-11-8::2.5::HTS,BRD-A02743701-001-03-6::2.5::HTS,BRD-A02759312-003-24-5::2.5::HTS,BRD-A02990301-003-06-6::2.5::HTS,BRD-A03061970-003-07-3::2.5::HTS,BRD-A03216249-003-24-3::2.5::HTS,BRD-A03359064-001-02-3::2.5::HTS,BRD-A03506276-001-01-5::2.5::HTS,BRD-A03623303-045-09-5::2.5::HTS,BRD-A03880619-001-01-0::2.5::HTS,BRD-A03932035-004-04-3::2.5::HTS,BRD-A04203391-001-08-3::2.5::HTS,BRD-A04252265-003-01-4::2.5::HTS,BRD-A04322457-003-17-9::2.5::HTS,BRD-A04327189-001-15-1::2.5::HTS,BRD-A04327189-003-03-3::2.5::HTS,BRD-A04352665-001-05-3::2.5::HTS,BRD-A04497688-001-04-2::2.5::HTS,...,BRD-K90563805-001-02-6::2.5::MTS004,BRD-K90629600-001-01-4::2.5::MTS004,BRD-K90982969-001-01-2::2.5::MTS004,BRD-K91244729-003-01-3::2.316613157::MTS004,BRD-K91405252-001-01-9::2.500076093::MTS004,BRD-K91543828-001-02-0::2.500044381::MTS004,BRD-K91900765-001-07-6::2.500030257::MTS004,BRD-K92073408-001-15-7::2.5::MTS004,BRD-K92413528-001-06-9::2.5::MTS004,BRD-K92426617-001-06-3::2.5::MTS004,BRD-K92490248-001-06-6::2.500037393::MTS004,BRD-K92581468-001-01-6::2.389082227::MTS004,BRD-K92902018-001-01-9::2.507952536::MTS004,BRD-K92908289-001-01-8::2.501017998::MTS004,BRD-K93442924-001-03-4::2.5::MTS004,BRD-K93900986-001-15-0::2.613957824::MTS004,BRD-K94072573-001-09-9::2.5::MTS004,BRD-K94146583-001-01-6::2.500085506::MTS004,BRD-K94830213-001-01-3::2.635005669::MTS004,BRD-K95362002-001-01-1::2.354783058::MTS004,BRD-K95941975-001-01-5::2.5::MTS004,BRD-K96206066-001-07-9::2.591957442::MTS004,BRD-K96344439-002-02-7::2.500011072::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96669468-300-01-1::2.5::MTS004,BRD-K96691275-001-01-2::2.5::MTS004,BRD-K96734425-304-01-7::2.500007729::MTS004,BRD-K96874295-015-01-0::2.500009134::MTS004,BRD-K97939847-001-12-0::2.5::MTS004,BRD-K98054437-001-09-9::2.494413182::MTS004,BRD-K98557884-001-01-6::2.5::MTS004,BRD-K99077012-001-01-9::2.332734192::MTS004,BRD-K99199077-001-16-1::2.603211317::MTS004,BRD-K99431849-001-01-7::2.500018158::MTS004,BRD-K99447003-335-04-1::2.37737659::MTS004,BRD-K99506538-001-03-8::2.5::MTS004,BRD-K99616396-001-05-1::2.499991421::MTS004,BRD-K99879819-001-02-1::2.5187366::MTS004,BRD-K99919177-001-01-3::2.5::MTS004,BRD-M63173034-001-03-6::2.64076472::MTS004
DepMap_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACH-000007,60,0,0,60,60,0,0,60,0,60,60,60,0,60,60,60,0,60,60,0,0,60,0,0,60,0,0,0,60,0,0,60,0,0,0,0,0,0,0,0,...,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,60,60,0,0,0,60,0,60,0,0,0,0,60,0,0
ACH-000011,0,49,0,49,49,0,49,0,0,49,0,0,49,0,0,0,0,0,0,49,49,0,49,49,0,0,49,49,0,49,0,0,49,49,0,0,49,0,0,0,...,49,0,0,0,49,49,49,0,49,0,0,0,0,49,0,0,49,0,49,0,49,49,49,49,49,0,0,0,0,0,49,49,0,49,0,0,0,49,49,0
ACH-000012,0,0,0,0,0,0,42,42,0,42,0,42,0,0,42,0,0,0,0,0,42,42,42,0,0,0,42,0,42,0,42,0,0,0,42,0,0,0,0,0,...,0,0,42,42,0,0,42,0,0,42,0,42,0,42,0,0,0,0,0,42,42,0,0,42,42,0,42,42,42,0,42,0,42,0,42,0,0,42,42,0
ACH-000013,0,0,0,37,0,0,37,0,0,0,0,0,0,37,37,0,0,0,0,37,37,0,0,37,37,37,0,0,37,0,0,0,37,0,0,0,0,0,37,0,...,0,0,0,0,0,0,0,0,37,0,0,0,0,37,0,0,0,0,0,37,0,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0,0,37
ACH-000014,0,46,0,0,46,0,0,0,0,0,46,46,0,0,0,0,46,0,0,46,0,0,46,46,46,0,46,0,46,46,46,0,0,0,46,46,46,46,46,46,...,0,0,0,0,0,46,0,0,0,0,0,0,0,0,0,

(404, 4686)

In [147]:
screen_rank_lt50_all_mul_col_sum = screen_rank_lt50_all_mul.sum(axis=0)

screen_rank_lt50_all_mul_col_sum.head()

screen_rank_lt50_pathogens_only_mul_col_sum = screen_rank_lt50_pathogens_only_mul.sum(axis=0)

screen_rank_lt50_pathogens_only_mul_col_sum.head()

BRD-A00077618-236-07-6::2.5::HTS     957411
BRD-A00100033-001-08-9::2.5::HTS     919258
BRD-A00147595-001-01-5::2.5::HTS     964181
BRD-A00218260-001-03-4::2.5::HTS    1003173
BRD-A00376169-001-01-6::2.5::HTS     896525
dtype: int64

BRD-A00077618-236-07-6::2.5::HTS    8622
BRD-A00100033-001-08-9::2.5::HTS    8374
BRD-A00147595-001-01-5::2.5::HTS    8740
BRD-A00218260-001-03-4::2.5::HTS    9230
BRD-A00376169-001-01-6::2.5::HTS    8116
dtype: int64

In [148]:
depmap_rank_lt50_sums_sum
pathogen_depmap_sums_sum

2200126

19783

In [ ]:
screen_rank_lt50_pathogens_only_mul_col_sum.index

In [149]:
#df_merged = pd.merge(screen_rank_lt50_pathogens_only_mul_col_sum, screen_rank_lt50_all_mul_col_sum, left_index=True, right_index=True)

df_merged = pd.concat([screen_rank_lt50_pathogens_only_mul_col_sum, screen_rank_lt50_all_mul_col_sum], axis=1)
df_merged.head()

,0,1
BRD-A00077618-236-07-6::2.5::HTS,8622,957411
BRD-A00100033-001-08-9::2.5::HTS,8374,919258
BRD-A00147595-001-01-5::2.5::HTS,8740,964181
BRD-A00218260-001-03-4::2.5::HTS,9230,1003173
BRD-A00376169-001-01-6::2.5::HTS,8116,896525


In [56]:
# demo

import scipy.stats as stats
table = np.array([[  392,  39948],[  3543, 392383]])

oddsratio, pvalue = stats.fisher_exact(table)
print("OddsR: ", oddsratio, "p-Value:", pvalue)

OddsR:  1.0867510207526063 p-Value: 0.12160419002760908


In [ ]:
import scipy.stats as stats

def odds_ratio(x):
  oddsratio, pvalue = stats.fisher_exact([ [x[0], x[1]], [pathogen_depmap_sums_sum, depmap_rank_lt50_sums_sum]])

  return str(oddsratio) + "__" + str(pvalue)

df_final = df_merged.apply(odds_ratio, axis=1)

#df_final = pd.concat([df_merged, df_merged.apply(lambda s: pd.Series({'feature1':s+1, 'feature2':s-1}))], axis=1)


df_final.head()

In [65]:
df_final_ = df_final
drugs0 = drugs[['column_name', 'name']]
df_final = df_final.to_frame()
df_final['BRD_ID'] = df_final.index

df_final0 = df_final.merge(drugs0, how='left', left_on='BRD_ID', right_on='column_name')

In [66]:
df_final0.head()

,0,BRD_ID,column_name,name
0,1.0867510207526063__0.12160419002760908,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00077618-236-07-6::2.5::HTS,8-bromo-cGMP
1,1.034581270676339__0.580094855665435,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,nifurtimox
2,0.9383081938132541__0.3336418588867909,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,balaglitazone
3,1.0852956838105765__0.14139657706634287,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,flutrimazole
4,0.9681617402822286__0.6337495507359023,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,KBG


In [85]:
df_final0[['odds','pval']] = df_final0[0].str.split("__", expand=True) 

df_final0 = df_final0.sort_values(by='odds', ascending=False)
df_final0.head()

,0,BRD_ID,column_name,name,odds,pval
0,1.0867510207526063__0.12160419002760908,BRD-A00077618-236-07-6::2.5::HTS,BRD-A00077618-236-07-6::2.5::HTS,8-bromo-cGMP,1.0867510207526063,0.12160419002760908
1,1.034581270676339__0.580094855665435,BRD-A00100033-001-08-9::2.5::HTS,BRD-A00100033-001-08-9::2.5::HTS,nifurtimox,1.034581270676339,0.580094855665435
2,0.9383081938132541__0.3336418588867909,BRD-A00147595-001-01-5::2.5::HTS,BRD-A00147595-001-01-5::2.5::HTS,balaglitazone,0.9383081938132541,0.3336418588867909
3,1.0852956838105765__0.14139657706634287,BRD-A00218260-001-03-4::2.5::HTS,BRD-A00218260-001-03-4::2.5::HTS,flutrimazole,1.0852956838105765,0.14139657706634287
4,0.9681617402822286__0.6337495507359023,BRD-A00376169-001-01-6::2.5::HTS,BRD-A00376169-001-01-6::2.5::HTS,KBG,0.9681617402822286,0.6337495507359023


In [75]:
! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1_Te26ihKLQ7TyK_XjlLs4jK4k_Kx-PLy' 
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('drug_calculated_properties.csv')
drugbank_calc = pd.read_csv('drug_calculated_properties.csv', sep=',')

file_id = '1b4NnapewD4ai3zLMUJ-PPUngvCDCXGOJ' 
downloaded = drive.CreateFile({'id': file_id})
# allows you to temporarily load your file in the notebook VM

downloaded.GetContentFile('drug.csv')
drugbank = pd.read_csv("drug.csv", sep=',')

In [76]:
drugbank_calc_ = drugbank_calc

In [77]:
drugbank_ = drugbank

In [69]:
drugbank.head()
drugbank_calc.head()

drugbank_merged = drugbank.merge(drugbank_calc, on='left', left_on='primary_key', right_on='parent_key')

drugbank_merged = drugbank.groupby

,primary_key,other_keys,type,created,updated,name,description,cas_number,unii,average_mass,monoisotopic_mass,state,synthesis_reference,fda_label,msds
0,DB00001,BTD00024;BIOD00024,biotech,2005-06-13,2020-06-12,Lepirudin,Lepirudin is identical to natural hirudin exce...,138068-37-8,Y43GF64R34,NaN,NaN,liquid,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...
1,DB00002,BTD00071;BIOD00071,biotech,2005-06-13,2020-07-01,Cetuximab,Cetuximab is an epidermal growth factor recept...,205923-56-4,PQX0D8J21J,NaN,NaN,liquid,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...
2,DB00003,BTD00001;BIOD00001,biotech,2005-06-13,2020-07-01,Dornase alfa,Dornase alfa is a biosynthetic form of human d...,143831-71-4,953A26OA1Y,NaN,NaN,liquid,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,//s3-us-west-2.amazonaws.com/drugbank/msds/DB0...
3,DB00004,BTD00084;BIOD00084,biotech,2005-06-13,2020-06-12,Denileukin diftitox,A recombinant DNA-derived cytotoxic protein co...,173146-27-5,25E79B5CTM,NaN,NaN,liquid,NaN,//s3-us-west-2.amazonaws.com/drugbank/fda_labe...,NaN
4,DB00005,BTD00052;BIOD00052,biotech,2005-06-13,2020-07-01,Etanercept,Dimeric fusion protein consisting of the extra...,185243-69-0,OP401G7OJC,NaN,NaN,liquid,"Timothy D. Osslund, Christi L. Clogston, Shon ...",NaN,NaN


,kind,value,source,parent_key
0,logP,-0.76,ALOGPS,DB00006
1,logS,-4.7,ALOGPS,DB00006
2,Water Solubility,4.64e-02 g/l,ALOGPS,DB00006
3,logP,-14,ChemAxon,DB00006
4,IUPAC Name,(4S)-4-[(2S)-2-[(2S)-2-[(2S)-2-{2-[(2S)-2-(2-{...,ChemAxon,DB00006


In [78]:
#drugbank_calc_ = drugbank_calc

drugbank_calc_pka_logP = drugbank_calc[(drugbank_calc['kind'] == 'logP') | (drugbank_calc['kind'] == 'pKa (strongest basic)')]
drugbank_calc_pka_logP.head()



,kind,value,source,parent_key
0,logP,-0.76,ALOGPS,DB00006
3,logP,-14,ChemAxon,DB00006
19,pKa (strongest basic),11.88,ChemAxon,DB00006
26,logP,1.04,ALOGPS,DB00007
29,logP,-2.4,ChemAxon,DB00007


In [90]:
drugbank_calc_pka_logP.dtypes

kind          object
value         object
source        object
parent_key    object
dtype: object

In [93]:
drugbank_calc_pka_logP['value'] = drugbank_calc_pka_logP['value'].astype('float')
drugbank_calc_pka_logP_pivot = drugbank_calc_pka_logP.pivot_table(index='parent_key',columns='kind',values='value', aggfunc='mean')
drugbank_calc_pka_logP_pivot.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


kind,parent_key,logP,pKa (strongest basic)
0,DB00006,-7.38,11.88
1,DB00007,-0.68,11.92
2,DB00014,-2.45,10.82
3,DB00027,5.17,NaN
4,DB00035,-3.55,11.77


In [97]:
drugbank0 = drugbank[['primary_key', 'name']]

drugbank_calc_pka_logP_pivot0 = drugbank_calc_pka_logP_pivot.merge(drugbank0, how='left', left_on='parent_key', right_on='primary_key')
drugbank_calc_pka_logP_pivot0.head()

,parent_key,logP,pKa (strongest basic),primary_key,name
0,DB00006,-7.38,11.88,DB00006,Bivalirudin
1,DB00007,-0.68,11.92,DB00007,Leuprolide
2,DB00014,-2.45,10.82,DB00014,Goserelin
3,DB00027,5.17,NaN,DB00027,Gramicidin D
4,DB00035,-3.55,11.77,DB00035,Desmopressin


In [98]:
df_final0_merged = df_final0.merge(drugbank_calc_pka_logP_pivot0, how='left', left_on=df_final0['name'].str.lower(), right_on=drugbank_calc_pka_logP_pivot0['name'].str.lower())

df_final0_merged.head()


,key_0,0,BRD_ID,column_name,name_x,odds,pval,parent_key,logP,pKa (strongest basic),primary_key,name_y
0,methoxsalen,1.570000272206189__4.272044417692328e-13,BRD-K63430059-001-17-2::2.5::HTS,BRD-K63430059-001-17-2::2.5::HTS,methoxsalen,1.570000272206189,4.272044417692328e-13,DB00553,1.940,-2.9,DB00553,Methoxsalen
1,bakuchiol,1.5598422598816155__1.8250257170092502e-12,BRD-K09785578-001-01-2::2.5::HTS,BRD-K09785578-001-01-2::2.5::HTS,bakuchiol,1.5598422598816155,1.8250257170092502e-12,NaN,NaN,NaN,NaN,NaN
2,ipratropium,1.4757511672145793__5.77831731141155e-11,BRD-A47449001-004-13-7::2.5::HTS,BRD-A47449001-004-13-7::2.5::HTS,ipratropium,1.4757511672145793,5.77831731141155e-11,DB00332,-0.455,-2.7,DB00332,Ipratropium
3,nitazoxanide,1.4415585765091017__1.768669495591181e-10,BRD-K96471533-001-09-0::2.592774962::MTS004,BRD-K96471533-001-09-0::2.592774962::MTS004,nitazoxanide,1.4415585765091017,1.768669495591181e-10,DB00507,2.130,-4.2,DB00507,Nitazoxanide
4,cyanopindolol,1.4398454796648765__6.427667478188251e-10,BRD-A47884604-051-01-2::2.5::HTS,BRD-A47884604-051-01-2::2.5::HTS,cyanopindolol,1.4398454796648765,6.427667478188251e-10,NaN,NaN,NaN,NaN,NaN


In [99]:
df = df_final0_merged[['name_x', 'odds', 'pval', 'pKa (strongest basic)', 'logP']]

df.rename(columns = {'odds' : 'odds_ratio', 'pKa (strongest basic)':'pka'}, inplace = True)
df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name_x,odds_ratio,pval,pka,logP
0,methoxsalen,1.570000272206189,4.272044417692328e-13,-2.9,1.940
1,bakuchiol,1.5598422598816155,1.8250257170092502e-12,NaN,NaN
2,ipratropium,1.4757511672145793,5.77831731141155e-11,-2.7,-0.455
3,nitazoxanide,1.4415585765091017,1.768669495591181e-10,-4.2,2.130
4,cyanopindolol,1.4398454796648765,6.427667478188251e-10,NaN,NaN


In [104]:
df_ = df

df.dtypes

df['pval'] = df['pval'].astype(float)
df['odds_ratio'] = df['odds_ratio'].astype(float)

df.dtypes
#df = df.astype(float)



name_x         object
odds_ratio     object
pval           object
pka           float64
logP          float64
dtype: object

name_x         object
odds_ratio    float64
pval          float64
pka           float64
logP          float64
dtype: object

In [105]:
df = df.sort_values(by='odds_ratio', ascending=False)
df.head()

,name_x,odds_ratio,pval,pka,logP
0,methoxsalen,1.570000,4.272044e-13,-2.9,1.940
1,bakuchiol,1.559842,1.825026e-12,NaN,NaN
2,ipratropium,1.475751,5.778317e-11,-2.7,-0.455
3,nitazoxanide,1.441559,1.768669e-10,-4.2,2.130
4,cyanopindolol,1.439845,6.427667e-10,NaN,NaN


In [107]:
df = df.dropna()

In [117]:
all = df.shape[0]
all
print("\n")
all_cads = df[(df['pka'] < 11) & (df['pka'] > 6.5) & (df['logP'] > 2)].shape[0]
all_cads
print("\n")
all_sig = df[(df['pval'] <= 0.005) & (df['odds_ratio'] >= 1) ].shape[0]
all_sig
print("\n")
all_sig_cads = df[(df['pval'] <= 0.005) & (df['odds_ratio'] >= 1) & (df['pka'] < 11) & (df['pka'] > 6.5) & (df['logP'] > 2)].shape[0]
all_sig_cads
print("\n")
table = np.array([[  all_sig_cads,  all_sig],[  all_cads, all]])

oddsratio, pvalue = stats.fisher_exact(table)
print("OddsR: ", oddsratio, "p-Value:", pvalue)




2197

594

139

46



OddsR:  1.2240147276118498 p-Value: 0.267260544782076


In [60]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [150]:
df.to_csv('/drive/My Drive/WashU/Data/SSRIs/FOR_PAPER/CRISPRscreens_drug_ID/df_CRISPR_all_vs_pathogens_top50.csv', index=False)